In [ ]:
from selenium import webdriver 
from bs4 import BeautifulSoup
import time
import csv
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

path = r'C:\Users\HP830G5\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe'
csv_file = r"C:\Users\HP830G5\Music\cars_price_prediction\data\scrapped\avito_cars_scrapping.csv"

labels = ["Date Pub", "Ville", "Année-Modèle", "Boite de vitesses", "Type de carburant",
          "Kilométrage", "Marque", "Modèle", "Nombre de portes",
          "Origine", "Première main", "Puissance fiscale", "État", "Prix"]

def avito_scraper(page_num):
    website = f'https://www.avito.ma/fr/maroc/v%C3%A9hicules?o={page_num}'
    service = Service(executable_path=path)
    driver = webdriver.Chrome(service=service)
    driver.get(website)
    time.sleep(5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    links = soup.find_all('a', class_='sc-1jge648-0 jZXrfL')
    print(f"Page {page_num} : {len(links)} annonces trouvées.")

    urls, prix_list, date_pub = [], [], []

    for link in links:
        if "/voitures_d_occasion/" in link.get('href', ''):
            urls.append(link['href'])
            prix_tag = link.find('span', class_='sc-3286ebc5-2 PuYkS')
            prix_list.append(prix_tag.text.strip() if prix_tag else None)
            date_tag = link.find('p', class_='sc-1x0vz2r-0 layWaX')
            date_pub.append(date_tag.text.strip() if date_tag else None)

    data = []

    for i in range(len(urls)):
        driver.get(urls[i])
        try:
            bouton = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((By.XPATH, "//button[@aria-label='Voir plus']"))
            )
            driver.execute_script("arguments[0].scrollIntoView(true);", bouton)
            driver.execute_script("arguments[0].click();", bouton)
            time.sleep(2)
        except:
            pass

        html_link = driver.page_source
        soup = BeautifulSoup(html_link, 'lxml')
        blocs = soup.find_all('div', class_='sc-cd1c365e-2 bToMqV')

        infos = {label: None for label in labels}
        infos["Date Pub"] = date_pub[i] if i < len(date_pub) else None
        infos["Prix"] = f"{prix_list[i]} DH" if i < len(prix_list) and prix_list[i] else None

        # Récupérer la ville
        ville_tag = soup.find('span', class_='sc-9ca53b09-15 eTxoKb')
        if ville_tag:
            ville_text = ville_tag.text.strip()
            if ',' in ville_text:
                infos["Ville"] = ville_text.split(',')[1].strip()
            else:
                infos["Ville"] = ville_text
        else:
            infos["Ville"] = None

        # Parcours des blocs pour remplir les autres informations
        for bloc in blocs:
            spans = bloc.find_all('span')
            if len(spans) == 2:
                valeur = spans[0].get_text(strip=True)
                label = spans[1].get_text(strip=True)
                if label in infos:
                    infos[label] = valeur
                    

        # On ajoute seulement si un prix est disponible
        if infos["Prix"]:
            data.append(infos)

    print(f"{len(data)} lignes valides trouvées sur cette page.")

    driver.quit()

    # Écriture dans le CSV
    if data:
        with open(csv_file, "a", newline='', encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=labels)
            if f.tell() == 0:
                writer.writeheader()
            writer.writerows(data)

if __name__ == '__main__':
    for i in range(40, 150):
        print(f"Scraping page {i}...")
        avito_scraper(i)
        time.sleep(10)


Scraping page 40...
Page 40 : 36 annonces trouvées.
